See [schema](https://oss-augur.readthedocs.io/en/dev/_images/schema.png) for details.

In [1]:
import psycopg2
import pandas as pd 
# from sqlalchemy.types import Integer, Text, String, DateTime
import sqlalchemy as s
import matplotlib
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import json

with open("config.json") as config_file:
    config = json.load(config_file)

database_connection_string = 'postgres+psycopg2://{}:{}@{}:{}/{}'.format(config['user'], config['password'], config['host'], config['port'], config['database'])

dbschema='augur_data'
engine = s.create_engine(
    database_connection_string,
    connect_args={'options': '-csearch_path={}'.format(dbschema)})

In [2]:

repo_info_query = f"""
SELECT
   repo_name, repo_id
FROM
   repo
WHERE
   repo_group_id = 25155;
    """
repo_info = pd.read_sql_query(repo_info_query, con=engine)
print(repo_info[0:100])

              repo_name  repo_id
0         rphash-golang    25506
1                aspect    25511
2                  psi4    25454
3           SimVascular    25521
4          purdue-fastr    25439
..                  ...      ...
87                OSHUN    25503
88  QuickPIC-OpenSource    25504
89              panache    25453
90                draft    25474
91                Block    25485

[92 rows x 2 columns]


In [3]:
repo_info_query = f"""
select * from "augur_operations"."worker_history" where job_model = 'pull_requests' 
and repo_id = 25512 
order by 
timestamp desc, repo_id;
"""
repo_info = pd.read_sql_query(repo_info_query, con=engine)
print(repo_info)

   history_id  repo_id                             worker      job_model  \
0      101664    25512  workers.pull_request_worker.47680  pull_requests   

   oauth_id           timestamp status  total_results  
0      1001 2021-02-10 10:12:27  Error              1  


In [4]:
import datetime 

current = datetime.date.today()
today = "'" + str(current) + "'"
#print(current)

first_current = current.replace(day=1)
last_month = first_current - datetime.timedelta(days=1)
end_date = "'" + str(last_month) + "'"

start = last_month - datetime.timedelta(days=180)
start_date = "'" + str(start) + "'"

def get_commits_by_repo(start_date, end_date, engine):
    import pandas as pd

    repo_list_commits = pd.DataFrame()
    repo_list_commits_query = f"""
            SELECT COUNT(DISTINCT commits.cmt_commit_hash), repo.repo_id, repo.repo_name, repo.repo_path from repo, commits
            WHERE 
                repo.repo_id = commits.repo_id
                AND repo.repo_path LIKE '%%OpenMP%%'
                AND commits.cmt_author_timestamp >= {start_date}
                AND commits.cmt_author_timestamp <= {end_date}
                AND cmt_author_name NOT LIKE '%%utomation%%'
                AND cmt_author_name NOT LIKE '%%ipeline%%'
                AND cmt_author_name NOT LIKE '%%Cloud Foundry%%'
                AND cmt_author_name NOT LIKE 'snyk%%'
                AND cmt_author_name NOT LIKE '%%bot'
                AND cmt_author_name NOT LIKE 'dependabot%%'
                AND cmt_author_name NOT LIKE '%%Bot'
                AND cmt_author_name NOT LIKE '%%BOT'
                AND cmt_author_name != 'cfcr'
                AND cmt_author_name != 'CFCR'
                AND cmt_author_name != 'Travis CI'
                AND cmt_author_name != 'Cloud Foundry London'
                AND cmt_author_name != 'pivotal-rabbitmq-ci'
                AND cmt_author_name != 'Bitnami Containers'
                AND cmt_author_name != 'Spring Operator'
                AND cmt_author_name != 'Spring Buildmaster'
            GROUP BY repo.repo_id
            ORDER BY COUNT(DISTINCT commits.cmt_commit_hash);
            """
    repo_list_commits = pd.read_sql_query(repo_list_commits_query, con=engine)
    
    return repo_list_commits
    
repo_list_commits = get_commits_by_repo(start_date, end_date, engine)

print(repo_list_commits)

Empty DataFrame
Columns: [count, repo_id, repo_name, repo_path]
Index: []


In [5]:
repo_list = pd.DataFrame()
repo_list_query = f"""
SELECT repo_id, repo_name, repo_path from repo
WHERE repo_name = 'BiG-CZ' OR repo_name = 'ADIOS' or repo_name = 'diana-hep' or repo_name = 'FingerPrint' or
      repo_name = 'ipop-project' or repo_name = 'open-source-dataturbine';
    """
repo_list = pd.read_sql_query(repo_list_query, con=engine)
print(repo_list)


   repo_id                repo_name                  repo_path
0    25498                   BiG-CZ                github.com/
1    25486                    ADIOS      github.com/ornladios/
2    25515                diana-hep                github.com/
3    25476              FingerPrint  github.com/rocksclusters/
4    25490             ipop-project                github.com/
5    25473  open-source-dataturbine        bitbucket.org/OSDT/


In [6]:
missing_query = f"""
SELECT
    * 
FROM
    (
    SELECT
        repo_info.repo_id,
        repo.repo_name,
repo.repo_git, 
        MAX ( pull_request_count ) AS max_pr_count,
        COUNT ( * ) AS meta_count 
    FROM
        repo_info,
        repo -- WHERE issues_enabled = 'true' 
    WHERE
        pull_request_count >= 1
        AND repo.repo_id = repo_info.repo_id 
    GROUP BY
        repo_info.repo_id,
        repo.repo_name, 
 repo.repo_git 
    ORDER BY
        repo_info.repo_id,
        repo.repo_name 
    ) yy
    LEFT OUTER JOIN (
    SELECT A
        .repo_id,
        A.repo_name,
        b.pull_request_count,
        d.repo_id AS pull_request_repo_id,
        e.last_collected,
        f.last_pr_collected,
        COUNT ( * ) AS pull_requests_collected,
        ( b.pull_request_count - COUNT ( * ) ) AS pull_requests_missing,
        ABS ( CAST ( ( COUNT ( * ) ) + 1 AS DOUBLE PRECISION ) / CAST ( b.pull_request_count + 1 AS DOUBLE PRECISION ) ) AS ratio_abs,
        ( CAST ( ( COUNT ( * ) ) + 1 AS DOUBLE PRECISION ) / CAST ( b.pull_request_count + 1 AS DOUBLE PRECISION ) ) AS ratio_prs 
    FROM
        augur_data.repo A,
        augur_data.pull_requests d,
        augur_data.repo_info b,
        ( SELECT repo_id, MAX ( data_collection_date ) AS last_collected FROM augur_data.repo_info GROUP BY repo_id ORDER BY repo_id ) e,
        ( SELECT repo_id, MAX ( data_collection_date ) AS last_pr_collected FROM augur_data.pull_requests GROUP BY repo_id ORDER BY repo_id ) f 
    WHERE
        A.repo_id = b.repo_id 
        AND LOWER ( A.repo_git ) LIKE '%%github.com%%' 
        AND A.repo_id = d.repo_id 
        AND b.repo_id = d.repo_id 
        AND e.repo_id = A.repo_id 
        AND b.data_collection_date = e.last_collected 
        AND f.repo_id = A.repo_id -- AND d.pull_request_id IS NULL
        
    GROUP BY
        A.repo_id,
        d.repo_id,
        b.pull_request_count,
        e.last_collected,
        f.last_pr_collected 
    ORDER BY
        ratio_abs desc
    ) zz ON yy.repo_id = zz.repo_id 
ORDER BY
    ratio_abs;
    """
missing_data = pd.read_sql_query(missing_query, con=engine)
print(missing_data)

    repo_id            repo_name  \
0     25494               lapack   
1     25431               mbuild   
2     25468              cctools   
3     25447                 quda   
4     25448              boinc-1   
5     25451              landlab   
6     25458             Trilinos   
7     25454                 psi4   
8     25497                 ompi   
9     25457               dealii   
10    25474                draft   
11    25475          concepts-ts   
12    25476          FingerPrint   
13    25478           awp-odc-os   
14    25479                  bbp   
15    25480             hercules   
16    25481                UCVMC   
17    25483              pegasus   
18    25485                Block   
19    25486                ADIOS   
20    25493              titan2d   
21    25521          SimVascular   
22    25438                fastr   
23    25439         purdue-fastr   
24    25450                 sage   
25    25459             libflame   
26    25460             arch

In [7]:
df = missing_data.loc[missing_data['ratio_abs'] < .95]
#print(df)
df1 = df[['repo_id', 'repo_git', 'last_pr_collected', 'pull_requests_missing']]
print(df1)
df1.to_csv('output/missing.csv')

Empty DataFrame
Columns: [repo_id, repo_id, repo_git, last_pr_collected, pull_requests_missing]
Index: []


In [8]:
table_query = f"""
SELECT
   tablename
FROM
   pg_catalog.pg_tables
WHERE
   schemaname = 'augur_data';
    """
tables = pd.read_sql_query(table_query, con=engine)
print(tables)

               tablename
0           analysis_log
1   chaoss_metric_status
2         commit_parents
3     pull_request_teams
4      pull_request_meta
..                   ...
66                issues
67          issue_labels
68     issue_message_ref
69          issue_events
70  contributors_history

[71 rows x 1 columns]


In [9]:
desc_table_query = f"""
SELECT
   COLUMN_NAME
FROM
   information_schema.COLUMNS
WHERE
   TABLE_NAME = 'repo_groups';
    """
table_desc = pd.read_sql_query(desc_table_query, con=engine)
print(table_desc)


             column_name
0   data_collection_date
1             rg_recache
2       rg_last_modified
3          repo_group_id
4           tool_version
5            data_source
6                rg_type
7            tool_source
8                rg_name
9         rg_description
10            rg_website


In [10]:
get_id_query = f"""
SELECT
   rg_name, repo_group_id
FROM
   repo_groups;
    """
get_id_desc = pd.read_sql_query(get_id_query, con=engine)
print(get_id_desc.sort_values('rg_name'))

         rg_name  repo_group_id
0  Howison Repos          25155


In [11]:
get_id_query = f"""
SELECT
   cntrb_login, cntrb_email, cntrb_canonical, cntrb_full_name, gh_login 
FROM
   contributors
WHERE
   gh_login = 'sgoggins' or cntrb_email = 's@goggins.com';
    """
get_id_desc = pd.read_sql_query(get_id_query, con=engine)
print(get_id_desc)

Empty DataFrame
Columns: [cntrb_login, cntrb_email, cntrb_canonical, cntrb_full_name, gh_login]
Index: []


In [12]:
get_id_query = f"""
SELECT cmt_author_name, cmt_author_email 
FROM commits 
WHERE cmt_author_name LIKE '%%Richard Johnson%%'
GROUP BY cmt_author_email, cmt_author_name;
"""
get_id_desc = pd.read_sql_query(get_id_query, con=engine)
print(get_id_desc)

Empty DataFrame
Columns: [cmt_author_name, cmt_author_email]
Index: []


In [13]:
get_id_query = f"""
SELECT commits.cmt_author_name, commits.cmt_author_email, commits.repo_id, 
       contributors.cntrb_canonical, contributors.cntrb_full_name
FROM commits, contributors 
WHERE commits.repo_id = 25541
    and (commits.cmt_author_name like '%%i%%' or commits.cmt_author_name like 'bryanl' or commits.cmt_author_email = 'iam@smartic.us')
GROUP BY commits.cmt_author_email, commits.cmt_author_name, commits.repo_id, contributors.cntrb_canonical, contributors.cntrb_full_name;
"""
get_id_desc = pd.read_sql_query(get_id_query, con=engine)
print(get_id_desc)

#cntrb_canonical, cntrb_full_name 
#                           FROM contributors

Empty DataFrame
Columns: [cmt_author_name, cmt_author_email, repo_id, cntrb_canonical, cntrb_full_name]
Index: []


In [14]:
repo_info_query = f"""
SELECT
   count(repo_name), repo_path
FROM
   repo
GROUP BY
   repo_group_id, repo_path;
    """
repo_info = pd.read_sql_query(repo_info_query, con=engine)
print(repo_info)

    count                                 repo_path
0       1                     github.com/Paradigm4/
1       3                    github.com/mosdef-hub/
2       2  github.com/UCLA-Plasma-Simulation-Group/
3       2                         github.com/BOINC/
4       1                    bitbucket.org/chemora/
..    ...                                       ...
59      1              git-wip-us.apache.org/repos/
60      1                      github.com/wkbjerry/
61      1                       source.usc.edu/svn/
62      1                     github.com/CMU-Quake/
63      1                    github.com/scikit-hep/

[64 rows x 2 columns]


In [15]:
    repo_list_commits_query = f"""
            SELECT COUNT(DISTINCT commits.cmt_commit_hash), repo.repo_path, repo.repo_group_id from repo, commits
            WHERE 
                repo.repo_id = commits.repo_id
                AND commits.cmt_author_timestamp >= '2020-01-01'
                AND commits.cmt_author_timestamp <= '2020-04-15'
            GROUP BY repo.repo_group_id, repo.repo_path
            ORDER BY COUNT(DISTINCT commits.cmt_commit_hash);
            """
    repo_list_commits = pd.read_sql_query(repo_list_commits_query, con=engine)


In [16]:
repo_list_commits

count                              repo_path  repo_group_id
0       1                      github.com/flame/          25155
1       4                     github.com/scifio/          25155
2       5                    github.com/TITAN2D/          25155
3      11                   github.com/SCECcode/          25155
4      11                   github.com/GOMC-WSU/          25155
5      15                github.com/SimVascular/          25155
6      38           github.com/Reference-LAPACK/          25155
7      45                    github.com/SRI-CSL/          25155
8      62                 github.com/scikit-hep/          25155
9      72                    github.com/PRUNERS/          25155
10     77                    github.com/landlab/          25155
11    171  github.com/cooperative-computing-lab/          25155
12    174                 github.com/mosdef-hub/          25155
13    210                   github.com/melt-umn/          25155
14    266                       github.com/psi4/          25155
15    276                    github.com/lattice/          25155
16    321                github.com/geodynamics/          25155
17    356                   github.com/open-mpi/          25155
18    398                      github.com/BOINC/          25155
19    411                     github.com/oracle/          25155
20    503                 github.com/yt-project/          25155
21    843                github.com/pegasus-isi/          25155
22   1364                     github.com/dealii/          25155
23   1535                   github.com/trilinos/          25155
24   1982                   github.com/sagemath/          25155

In [17]:
all_commits_query = f"""
        SELECT DISTINCT(commits.cmt_commit_hash), repo.repo_id, repo.repo_group_id, repo.repo_name, repo.repo_path, 
            commits.cmt_author_email, commits.cmt_author_timestamp from repo, commits
        WHERE 
            repo.repo_id = commits.repo_id
        GROUP BY repo.repo_id, commits.cmt_commit_hash, commits.cmt_author_email, commits.cmt_author_timestamp
        ORDER BY repo.repo_id;
        """
all_commits = pd.read_sql_query(all_commits_query, con=engine)
# by_repo = all_commits.loc[all_commits.groupby('repo_id').cmt_author_timestamp.idxmax()].sort_values('cmt_author_timestamp')

In [18]:
all_commits

cmt_commit_hash  repo_id  repo_group_id  \
0       001dd26e0aa4662b298c0a44f281a016ff416b99    25430          25155   
1       003e7eb431b94df2fcb76b19eb80216f9ee98ae0    25430          25155   
2       0044f56b49ef33df95b8a6fdb4352ef02621f2e4    25430          25155   
3       00a64c696a260718450d8234bf3b7ee250724583    25430          25155   
4       00beaebdf175fd9462f70a304a55f05d5b9c525e    25430          25155   
...                                          ...      ...            ...   
498110  ff71e8b69a92c3af89cd2557f950ee354644e608    25521          25155   
498111  ff8066a6654f81a7b2e882300bc3968b000c8ced    25521          25155   
498112  ff9f87288531c9d5eed2845c466a0fb7cf88d670    25521          25155   
498113  ffcb2108af1be253a493429212b0a7572b03f992    25521          25155   
498114  ffd6f5a264a182cd6f5e4c772f4d13523630066a    25521          25155   

          repo_name                repo_path                 cmt_author_email  \
0             foyer   github.com/mosdef-hub/  raymat@master.cl.vanderbilt.edu   
1             foyer   github.com/mosdef-hub/   christoph.klein@vanderbilt.edu   
2             foyer   github.com/mosdef-hub/   christoph.klein@vanderbilt.edu   
3             foyer   github.com/mosdef-hub/   christoph.klein@vanderbilt.edu   
4             foyer   github.com/mosdef-hub/   christoph.klein@vanderbilt.edu   
...             ...                      ...                              ...   
498110  SimVascular  github.com/SimVascular/              gmaher2@hotmail.com   
498111  SimVascular  github.com/SimVascular/   osmsc@users.noreply.github.com   
498112  SimVascular  github.com/SimVascular/   osmsc@users.noreply.github.com   
498113  SimVascular  github.com/SimVascular/   adamupdegrove@Adams-iMac.local   
498114  SimVascular  github.com/SimVascular/   adamupdegrove@Adams-iMac.local   

             cmt_author_timestamp  
0       2019-06-12 18:50:38+02:00  
1       2016-01-14 06:56:34+01:00  
2       2017-02-09 19:05:48+01:00  
3       2017-02-13 18:59:39+01:00  
4       2017-03-09 17:39:36+01:00  
...                           ...  
498110  2019-04-25 01:34:37+02:00  
498111  2018-11-17 20:57:36+01:00  
498112  2018-01-13 09:31:16+01:00  
498113  2017-09-20 18:20:32+02:00  
498114  2017-07-13 17:55:34+02:00  

[498115 rows x 7 columns]

In [19]:
by_org = all_commits.loc[all_commits.groupby('repo_group_id').cmt_author_timestamp.idxmax()].sort_values('cmt_author_timestamp')
for index, row in by_org.iterrows():
    print 
    top_contribs = all_commits.loc[all_commits['repo_group_id'] == row.repo_group_id].cmt_author_email.value_counts()
    print(row.repo_path, top_contribs.index[0], top_contribs[0], top_contribs.index[1], top_contribs[1])

ValueError: 

In [20]:
desc_table_query = f"""
SELECT
   COLUMN_NAME
FROM
   information_schema.COLUMNS
WHERE
   TABLE_NAME = 'repo';
    """
table_desc = pd.read_sql_query(desc_table_query, con=engine)
print(table_desc)

                     column_name
0                        repo_id
1                  repo_group_id
2                     repo_added
3                       owner_id
4                     updated_at
5           data_collection_date
6                  repo_archived
7   repo_archived_date_collected
8                            url
9                   tool_version
10                   description
11              primary_language
12                    created_at
13                   forked_from
14                   data_source
15                      repo_git
16                     repo_path
17                     repo_name
18                   tool_source
19                   repo_status
20                     repo_type
